# Extract and save as JSON

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import glob
import re
import os
import json
import locale
locale.setlocale(locale.LC_TIME, "de_ch")

'de_ch'

In [2]:
path_in = '../data/article/'
path_out = '../data_cleaned/article/'

In [6]:
def one_or_warn(selector):
    l = soup.select(selector)
    if(len(l) > 1):
        print("More than one %s at %s" % (selector, f))
    if(len(l) > 0):
        return l[0].string
    return ""

reg_abbr = re.compile('\((([^)]*))\)[^(]*$', re.IGNORECASE)

records = []
filelist = glob.glob("%s*.html" % path_in)
for f in filelist:
    #f = '../data/article/domkapitelleaks-was-martin-werlen-dem-domkapitel-mitteilte.html'
    
    # Get ID
    id = os.path.basename(f).replace('.html', '')
    
    # Check if exists
    if os.path.isfile('%s%s.json' % (path_out, id)):
        continue
        
    print(f)
    
    record = {'id': id}
    
    soup = BeautifulSoup(open(f, encoding='utf-8'), "html.parser")

    # Find Tag
    record['tag'] = one_or_warn('.esf-img-tag span')
    
    # Tag 2 (Gastkommentare falsch Kategorisiert)
    record['tag2'] = record['tag']
    lead = soup.select_one('.text p strong')
    if lead:
        if any(s.lower() in lead.text.lower() for s in ['Gastbeitrag', 'Gastkommentar', 'Kommentar']):
            record['tag2'] = 'Kommentar'

    # Find Title
    record['title'] = one_or_warn('h1')

    # Lead
    #record['lead'] = soup.select_one('.text p strong').string

    # Text
    record['text'] = soup.select_one('.text').text
    

    # Date
    icon = soup.select_one('img[src="https://www.kath.ch/wp-content/themes/cathkathcatt/images/icon_date.png"]')
    if(len(icon) == None):
        print("No date found at %s" % f)
    date_s = icon.parent.parent.select_one('.col-9').string.replace('\n', '').strip()
    date = date_s.split('|')[0].strip()
    time = date_s.split('|')[1].strip()

    d = datetime.strptime("%s %s" % (date, time), '%d. %B %Y %H:%M')
    record['date'] = d.strftime("%Y-%m-%d %H:%M")

    # Get author
    s = soup.select('.text>p')
    if len(s) > 2:
        s = s[1].text
        if(len(s.split(' ')) <= 6):

            # Looks like there is an author
            record['author'] = s.strip()

        else:

            # Try to guess abbrev.
            reg = reg_abbr.search(soup.select('.text>p')[-1].text)
            if reg:
                record['author'] = reg.group(1).strip()
                
    if ('autor' in record) == False:
        # Check, if Rauch in Lead
        lead = soup.select_one('.text p strong')
        if lead and ('raphael rauch' in soup.select_one('.text p strong').text.lower()):
            record['author'] = 'Raphael Rauch'
        
    # Save to file
    with open('%s%s.json' % (path_out, record['id']), 'w', encoding='UTF-8') as fp:    
        json.dump(record, fp, ensure_ascii=False)
        
    
print("finito")

../data/article\friede-statt-gewalt-birmanische-ordensfrau-erlaeutert-ihren-einsatz-fuer-demonstranten.html
../data/article\friedenstaube-statt-kalifat-mossul-feiert-papst-franziskus.html
../data/article\friedeseimitdir-10-tweets-die-mich-beruehrt-haben.html
../data/article\frische-winde-brauchen-wir-in-der-kirche-ein-merk-wuerdiger-brief-von-tarcisi-venzin-an-papst-franziskus.html
../data/article\frueherer-dominikaner-provinzial-hubert-niclasse-gestorben.html
../data/article\frueherer-papst-benedikt-xvi-vollendet-sein-94-lebensjahr.html
../data/article\frueherer-vatikanbank-chef-wegen-geldwaesche-verurteilt.html
../data/article\frueherer-vatikanischer-oekumene-minister-cassidy-gestorben.html
../data/article\fuenf-goldene-kroenchen-fuer-die-gottesmutter-und-das-jesuskind.html
../data/article\fuenf-jahre-bischof-joseph-maria-gut-holz.html
../data/article\fuenf-jahre-nach-havanna-gemeinsam-die-welt-nach-der-pandemie-vorbereiten.html
../data/article\fuenf-sutras-zum-prophetischen-wirken-v